# Simulated annealing search

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

In [3]:
import sys
sys.path.append('../common')
from utils import weight3 as weight_fn
from utils import bag_weight, score
from utils import MAX_WEIGHT, AVAILABLE_GIFTS, GIFT_TYPES, N_TYPES, N_BAGS

In [4]:
fixed_weights = {}
fixed_weights['ball'] = 1.99876912083
fixed_weights['bike'] = 20.0021364556
fixed_weights['blocks'] = 11.6630321858
fixed_weights['book'] = 2.00086596571
fixed_weights['coal'] = 23.7866257713
fixed_weights['doll'] = 4.9993625282
fixed_weights['gloves'] = 1.40310067709
fixed_weights['horse'] = 4.99527064522
fixed_weights['train'] = 10.0234458084
fixed_weights

{'ball': 1.99876912083,
 'bike': 20.0021364556,
 'blocks': 11.6630321858,
 'book': 2.00086596571,
 'coal': 23.7866257713,
 'doll': 4.9993625282,
 'gloves': 1.40310067709,
 'horse': 4.99527064522,
 'train': 10.0234458084}

In [5]:
n_bags_per_state = 1
empty_state = tuple([tuple([0]*N_TYPES)]*n_bags_per_state)

In [6]:
def fix_weight(gift_index):
    return fixed_weights[GIFT_TYPES[gift_index]]

def bag_fix_weight(bag):
    out = 0
    for i, c in enumerate(bag):
        out += fix_weight(i) * c
    return out

def state_fix_score(state):
    score = 0 
    for bag in state:
        score += bag_fix_weight(bag)
    return score

type_cost = {}
for g in fixed_weights:
    type_cost[g] = 1.0/fixed_weights[g]
#     type_cost[g] = (MAX_WEIGHT - fixed_weights[g]) / MAX_WEIGHT 
type_cost

{'ball': 0.5003079092920669,
 'bike': 0.04999465943149438,
 'blocks': 0.08574099634377431,
 'book': 0.49978360226900737,
 'coal': 0.04204043102265309,
 'doll': 0.2000255021233769,
 'gloves': 0.7127072321524197,
 'horse': 0.20018935329498216,
 'train': 0.09976609033611623}

In [7]:
# Penalize states that heavy than goal score 
def h1(state, limit):    
    w = state_fix_score(state)
    if w < limit:
        return (limit - w) / limit
    else:
        return 1.0
    
    
# Difference between fix / 'real' masses
def h2(state):
    w2 = state_fix_score(state)
    w1 = score(state)
    if w2 > 0:
        return abs(w2 - w1)*1.0 / w2
    else:
        return 1.0


# Count number of rejected bags :
def h3(state):
    _, rejected = score(state, return_rejected=True)
    return rejected

In [8]:
alpha = 0.7
goal_score = n_bags_per_state*MAX_WEIGHT*alpha
limit = goal_score + 0.5*(MAX_WEIGHT-goal_score)

initial_state = ((0,0,0,0,0,0,0,0,0,),)

In [11]:
def value(state):
    return h1(state, limit) + h2(state) + h3(state)

value(initial_state)

2.0

In [10]:
from scipy.optimize import basinhopping

In [39]:
np.random.randint(len(GIFT_TYPES), size=5)

array([4, 3, 0, 5, 2])

In [14]:
class SBP_take_step:
    def __init__(self, stepsize=5.0):
        self.stepsize = stepsize
    def __call__(self, x):
        n = np.ceil(self.stepsize)
        indices = np.random.randint(len(GIFT_TYPES), size=n)
        for index in indices:
            x[index] += 1
        return x

def SBP_func(state):
    return MAX_WEIGHT - score(state)

In [15]:
SBP_func(initial_state)

50.0

In [ ]:
minimizer_kwargs = {"method": "BFGS"}
result = basinhopping(SBP_func, initial_state, minimizer_kwargs=minimizer_kwargs, niter=200, take_step=SBP_take_step())
print("global minimum: x = %.4f, f(x0) = %.4f" % (result.x, result.fun))

In [35]:
score(result.x)

(22.928007158010217, 0.0)